# Twitter API
Sólo con cuenta de desarrollador API's que provee Twitter:

* Tweets : buscar, publicar, filtrar, transmitir, etc.
* Ads : análisis de audiencia
* DM : enviar y recibir, respuesta directa, mensajes de bienvenida, etc. (aún en prueba)
* Cuentas y usuarios : administración de cuentas e interacciones con usuarios
* Media : acceder y subir a la plataforma fotos, videos, gif's
* Trends : trending topics dada una localización
* Geo : información de lugares cerca de una localización

Librerías de Python para trabajar con la API de Twitter
* Tweepy (la más recomendada y utilizada)
* Python Twitter Tools
* Twython
* TwitterAPI
* TwitterSearch

## Recolección por medio de APIs
API (Aplication Programming Interface) es una biblioteca, un conjunto de funciones que ofrece una cierta aplicación para ser accesada por otra.

API Twiiter: tweepy Modo streaming

API REST (Representational State Transfer, transferencia de estado representacional) es un método sencillo para recibir y enviar datos en cualquier formato, XML, JSON. Para ello se utilizan funciones POST, GET, PUT, DELETE.

## RESTful API Cuenta de desarrollador de Twitter
Necesitamos:

Crear una APP en twitter, en la cual se generan las siguientes llaves: consumer key API-KEY consumer seceret key API-KEY Secret access token key access token secret key Instalar las librerías para trabajar con twitter desde python: tweepy

### conda install -c conda-forge tweepy

In [ ]:
import os 
import pandas as pd
import tweepy as tw

In [ ]:
#Crear un app, ir a Keys and Access Tokens (copiar)
#Click en "Create y accese Token Secret" (copiar)
#Credentials

consumer_key = "cI1rISlIU5FgB4Vcy1T7BYXHh"
consumer_secret = "hXW47PUQcHIfE7yoM1SN6CPFrcUwKZ4sfxqfOjjllxE9Wd7ehb"
access_token = "1503435140781031430-7ih42hjcdOioh4xbCc2N52uBsPiAOk"
access_token_secret = "Suf2DidgjWIkPPLAdLUvRo1u3FWg0F5fZ99TIacnyEfmV"

import json
creds = {}
creds["CONSUMER_KEY"] = ""
creds["CONSUMER_SECRET"] = ""
creds["ACCESS_TOKEN"] = ""
creds["ACCESS_TOKN_SECRET"] = ""
with open("twitter_creds.json", "w") as arch:
    json.dump(creds, arch)

In [ ]:
#se crea un objeto de la clase OAuthHandler para poder utilizar la API con python
auth = tw.OAuthHandler(consumer_key, consumer_secret) #objeto

#credenciales para utilizar la API, en
auth.set_access_token(access_token, access_token_secret)

api = tw.API(auth, wait_on_rate_limit=True)
#objeto de la clase API, que nos permitirá realizar las consulta

In [ ]:
#Ejemplo: dos variables de búsqueda
search_words = "#wildfires" #<- busqueda con este hashtag
date_since = "2018-11-16"

In [ ]:
#Colecta los tweets según nuestras variables
#Buscamos search_words, los tweets en inglés, desde la fecha date_since, se restringe la busqueda
#Se utiliza el método Cursor para obtener los tweets que contengan la busqueda en este caso #wildfire
tweets = tw.Cursor(api.search_tweets, q = search_words, lang = "en", since = date_since).items(5)
#Cursor también devuelve información de quien escribe el tweet y la fecha además del texto del
tweets

In [ ]:
#Despliega los tweets
for tweet in tweets:
    print(tweet.text)
#Otra manera:
#[tweet.text for tweet in tweets]

In [ ]:
#Eliminar los retweets
new_search=search_words + " -filter:retweets"
new_search

In [ ]:
tweets = tw.Cursor(api.search_tweets,q=new_search, lang="en",since=date_since).items(15)
[tweet.text for tweet in tweets]

In [ ]:
#Quien publica
tweets = tw.Cursor(api.search,q=new_search, lang="en",since=date_since).items(15)
users_locs = [[tweet.user.screen_name, tweet.user.location] for tweet in tweets]
users_locs

In [ ]:
tweet_text = pd.DataFrame(data=users_locs, columns=['user', "location"])
tweet_text

In [ ]:
# Nueva busqueda
new_search = "TecNM -filter:retweets"
tweets = tw.Cursor(api.search_tweets, q=new_search, lang="es", 
                   since="2020-01-01").items(10)
all_tweets = (tweet.text for tweet in tweets)
all_tweets

In [ ]:
tweets = tw.Cursor(api.search_tweets, q=new_search, lang="es",
                   since="2020-01-01").items(10)

users_locs = [[tweet.user.screen_name, tweet.user.location] 
              for tweet in tweets]

tweet_tecnm = pd.DataFrame(data=users_locs, 
                           columns=['user', 'location'])
tweet_tecnm

In [ ]:
#contando palabras
search_term = "#COVID -filter:retweets"
tweets = tw.Cursor(api.search, q=search_term, lang="en", 
                   since='2020-01-01').items(1000)

all_tweets = [tweet.text for tweet in tweets]
all_tweets[:5]

In [ ]:
import re
def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """

    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [ ]:
all_tweets_no_urls = [remove_url(tweet) for tweet in all_tweets]
all_tweets_no_urls[:5]

In [ ]:
all_tweets_no_urls[0].split()

In [ ]:
all_tweets_no_urls[0].lower().split()

In [ ]:
#Creamos una lista de listas que contengan las palabras en minuscula para cada tweets
words_in_tweet = [tweet.lower().split() for tweet in all_tweets_no_urls]
words_in_tweet[:2]

In [ ]:
#Vemos la frecuencia de cada palabra
#conda install -c conda-forge itertools
#conda install -c conda-forge textblob

import itertools #crea objetos iterables, es rápido
import collections #crea colecciones como un dict especial sino existe la palabra agrega al dict

# Lista de todas las palabras en todos los tweets
all_words_no_urls = list(itertools.chain(*words_in_tweet))

# Cuenta las palabras
counts_no_urls = collections.Counter(all_words_no_urls)

#Selecciona las 15 más utilizadas
counts_no_urls.most_common(15) #método de collection.Counter

In [ ]:
#utilizamos pandas
clean_tweets_no_urls = pd.DataFrame(counts_no_urls.most_common(15), columns=['words','count'])
clean_tweets_no_urls.head()

In [ ]:
import matplotlib.pyplot as plt

fig, ax=plt.subplots(figsize=(8,8))

#Grafica de barra horizontal
clean_tweets_no_urls.sort_values(by='count').plot.barh(x='words',y='count',ax=ax, color="purple")
ax.set_title("Palabras mas utilizadas en Tweets (search_words=#COVID )")
plt.show()